<a href="https://colab.research.google.com/github/Bulaque/Homes_Sales_HW/blob/main/Home_Sales_starter_code_colab_Mod22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,122 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,083 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restric

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
spark.sparkContext.addFile(url)
home_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

In [5]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [17]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

spark.sql("SELECT date_built, bedrooms, round(avg(price), 2) FROM home_sales WHERE bedrooms = 4 GROUP BY date_built, bedrooms ORDER BY date_built").show()

+----------+--------+--------------------+
|date_built|bedrooms|round(avg(price), 2)|
+----------+--------+--------------------+
|      2010|       4|           296800.75|
|      2011|       4|            302141.9|
|      2012|       4|           298233.42|
|      2013|       4|           299999.39|
|      2014|       4|           299073.89|
|      2015|       4|           307908.86|
|      2016|       4|           296050.24|
|      2017|       4|           296576.69|
+----------+--------+--------------------+



In [16]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

spark.sql("SELECT date_built, bedrooms, bathrooms, round(avg(price), 2) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built, bedrooms, bathrooms ORDER BY date_built").show()

+----------+--------+---------+--------------------+
|date_built|bedrooms|bathrooms|round(avg(price), 2)|
+----------+--------+---------+--------------------+
|      2010|       3|        3|           292859.62|
|      2011|       3|        3|           291117.47|
|      2012|       3|        3|           293683.19|
|      2013|       3|        3|           295962.27|
|      2014|       3|        3|           290852.27|
|      2015|       3|        3|            288770.3|
|      2016|       3|        3|           290555.07|
|      2017|       3|        3|           292676.79|
+----------+--------+---------+--------------------+



In [18]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

spark.sql("SELECT date_built, bedrooms, bathrooms, round(avg(price), 2) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built, bedrooms, bathrooms ORDER BY date_built").show()

+----------+--------+---------+--------------------+
|date_built|bedrooms|bathrooms|round(avg(price), 2)|
+----------+--------+---------+--------------------+
|      2010|       3|        3|           292859.62|
|      2011|       3|        3|           291117.47|
|      2012|       3|        3|           293683.19|
|      2013|       3|        3|           295962.27|
|      2014|       3|        3|           290852.27|
|      2015|       3|        3|            288770.3|
|      2016|       3|        3|           290555.07|
|      2017|       3|        3|           292676.79|
+----------+--------+---------+--------------------+



In [21]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("SELECT view, round(avg(price), 2) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.5808184146881104 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table home_sales")

DataFrame[]

In [24]:
# 8. Check if the table is cached.

spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("SELECT view, round(avg(price), 2) FROM home_sales WHERE price >= 350000 GROUP BY view ORDER BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 1.1409885883331299 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("part_home_sales")

In [27]:
# 11. Read the parquet formatted data.

p_df = spark.read.parquet('part_home_sales')

In [28]:
# 12. Create a temporary table for the parquet data.

p_df.createOrReplaceTempView('parquet_home_sales')

In [29]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("SELECT view, round(avg(price), 2) FROM parquet_home_sales WHERE price >= 350000 GROUP BY view ORDER BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 1.17099928855896 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table home_sales")

DataFrame[]

In [32]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False